<a href="https://colab.research.google.com/github/adamrhurley/FYP/blob/main/MusicRecommenderSystem/svd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b
# https://predictivehacks.com/how-to-run-recommender-systems-in-python/

In [ ]:
pip install surprise

     |████████████████████████████████| 11.8MB 248kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617584 sha256=fc248e99dcd71941d2917e188619d1c72f13e2065e9f15f04783bbefb5117192
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection.validation import cross_validate

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
users = pd.read_csv(io.BytesIO(uploaded['user_artists.dat']), sep='\t')
artists = pd.read_csv(io.BytesIO(uploaded['artists.dat']), sep='\t',usecols=['id','name'])

In [ ]:
mergedDF = pd.merge(
  artists, users,
  how="inner",
  left_on="id",
  right_on="artistID"
)
mergedDF.head()

,id,name,userID,artistID,weight
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152


In [ ]:
users.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [ ]:
artists.head()

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


In [ ]:
print(data)

In [ ]:
reader = Reader(rating_scale = (0,100000))

data = Dataset.load_from_df(users[['userID', 'artistID', 'weight']], reader)
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    99320.557199352.009699349.217599314.342799294.846799310.636599343.050199335.986499295.607299308.094499322.434820.2368 
MAE (testset)     99283.790599315.371999296.814399285.934499258.326099270.510299321.971599281.516899254.215699275.454199284.390520.9760 
Fit time          5.06    5.01    4.97    4.94    5.06    4.97    5.04    4.99    5.10    4.92    5.00    0.05    
Test time         0.07    0.20    0.07    0.21    0.08    0.07    0.11    0.07    0.07    0.08    0.10    0.05    


{'fit_time': (5.057538747787476,
  5.008737564086914,
  4.967878580093384,
  4.9387006759643555,
  5.0551979541778564,
  4.973504066467285,
  5.0354907512664795,
  4.994671583175659,
  5.098316431045532,
  4.918747425079346),
 'test_mae': array([99283.79049978, 99315.37187213, 99296.81430418, 99285.93440327,
        99258.32598272, 99270.5101799 , 99321.97145319, 99281.51679098,
        99254.21555194, 99275.4541346 ]),
 'test_rmse': array([99320.55709854, 99352.00955155, 99349.21745631, 99314.34274576,
        99294.84671263, 99310.63654119, 99343.05006664, 99335.98641778,
        99295.60718699, 99308.0944347 ]),
 'test_time': (0.0748589038848877,
  0.19835877418518066,
  0.0736534595489502,
  0.20742177963256836,
  0.07655024528503418,
  0.07345032691955566,
  0.11056733131408691,
  0.07344865798950195,
  0.07276463508605957,
  0.07508659362792969)}

In [ ]:
userDF = users[(users['userID'] == 35)]
userDF = userDF.set_index('artistID')
userDF = userDF.join(artists)['name']
userDF.head(userDF.shape[0])

artistID
186                     Elton John
212                        Man Man
444              She Wants Revenge
920                          Viper
964                          Hevia
1169                  Lunatic Soul
1170                  Billy Talent
1171               Escape The Fate
1172                   Silverstein
1173    The Red Jumpsuit Apparatus
1174                 You Me At Six
1175                  The Blackout
1176                       Alesana
1177                        Sum 41
1178             Dance Gavin Dance
Name: name, dtype: object

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
artists = artists.copy()

artists['EstimatePlays'] = artists['id'].apply(lambda x: svd.predict(35, x).est)
artists = artists.sort_values(by=['EstimatePlays'], ascending=False)

In [ ]:
artists.head(10000)

,id,name,EstimatePlays
0,1,MALICE MIZER,100000
11751,12177,Guilherma e santiago,100000
11757,12183,Thao with The Get Down Stay Down,100000
11756,12182,リア・ディゾン,100000
11755,12181,Maria Cecilia e Rodolfo - Mato e morro por você,100000
...,...,...,...
4107,4191,Hot Water Music,100000
4105,4189,Alkaline Trio,100000
3993,4069,The Dandy Warhols,100000
4104,4188,Jawbreaker,100000
